In [31]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.linear_model import LinearRegression
from IPython.display import display, HTML
from dash import Dash, dcc, html, Input, Output, callback
import statsmodels.api as sm

In [45]:
plt.close("all")
data = pd.read_csv(r"House_Info.csv")
data = data.reset_index()
data['Closing_Date'] = data['Closing_Date'].astype('datetime64[ns]')
list_price = data.List_Price
sale_price = data.Sold_Price

In [33]:
tableau_embed_code = "<div class='tableauPlaceholder' id='viz1689010969504' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Pl&#47;PlayGround_16883995622190&#47;HouseProfitablility&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='PlayGround_16883995622190&#47;HouseProfitablility' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Pl&#47;PlayGround_16883995622190&#47;HouseProfitablility&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-US' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1689010969504');                    var vizElement = divElement.getElementsByTagName('object')[0];                    if ( divElement.offsetWidth > 800 ) { vizElement.style.width='1000px';vizElement.style.height='827px';} else if ( divElement.offsetWidth > 500 ) { vizElement.style.width='1000px';vizElement.style.height='827px';} else { vizElement.style.width='100%';vizElement.style.height='1127px';}                     var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>"
display(HTML(tableau_embed_code))

### Washington County Price Comparison History
- Explain the background, use, and how to use the widget above.

In [37]:
fig1 = px.scatter(data, x="Closing_Date", y=data.Sold_Price - data.List_Price,
                 labels={
                 "Closing_Date": "Closing Date ",
                 "y" : "Pricing Differential "
                 }
                 )
fig1.update_layout(
title="Listing and Closing Price Differentials",
xaxis_title="Year",
yaxis_title="Pricing Differential",
paper_bgcolor="#fff",
plot_bgcolor="#e3e3e3",
font=dict(
    family="Courier New, monospace",
    size=18,
    color="#000000"
    )
)
fig1.update_layout(yaxis_range=[-600000,200000])

### Washington County Price Differential History
- Explain the background, use, and how to use the widget above.

In [35]:
fig2 = px.box(data,y="Sold_Price", x= data.Closing_Year)

fig2.update_layout(
    title="Sale Price Box",
    xaxis_title="Year",
    yaxis_title="Sale Price",
    paper_bgcolor="#fff",
    plot_bgcolor="#e3e3e3",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#000000"
        )
    )
fig2.update_layout(yaxis_range=[0,800000])

### Washington County Sale Price Statistics
- Explain the background, use, and how to use the widget above.

In [44]:
X = data.Closing_Year
X = sm.add_constant(X)  # Add a constant term to the independent variable
y = data.List_Price - data.Sold_Price

model = sm.OLS(y, X)
results = model.fit()


app = Dash(__name__)

app.layout = html.Div([
    html.H4('Linear Regression Testing for Price Difference'),
    dcc.Graph(id="scatter-plot"),
    html.P("Filter by year:"),
    dcc.RangeSlider(
        id='range-slider',
        min=2015, max=2022, step=.5,
        marks={2015: '2015', 2016: '2016', 2017: '2017', 2018: '2018', 2019: '2019',2020: '2020', 2021: '2022',2022: '2022'},
        value=[2015, 2022]
    ),
    dcc.Input(
            id="input1".format(_),
            type="number",
            placeholder="Year",
            min=2022, max=2045, step=1,
            value=2022),
    dcc.Input(
            id="input2".format(_),
            type="number",
            placeholder="Listing Price".format(_),
            value=0
        ),
    html.Div(id="output"),
])

@app.callback(
    Output("scatter-plot", "figure"),
    Input("range-slider", "value"))
def update_scatter_chart(slider_range):
     # replace with your own data source
    low, high = slider_range
    mask = (data['Closing_Year'] > low) & (data['Closing_Year'] < high)
    fig = px.scatter(
        # TODO: Change the Difference misspelling in CSV.
        data[mask], x="Closing_Date", y="Price_Difference",opacity=0.65,
        trendline='ols', trendline_color_override='darkblue', hover_data=['Sold_Price','List_Price'],
        labels={
                     "Closing_Date": "Closing Date ",
                     "Price_Difference" : "Pricing Differential ",
                     "List_Price" : "Listing Price ",
                     "Sold_Price" : "Selling Price"
                 }
        )
    return fig


@callback(
    Output("output", "children"),
    Input("input1", "value"),
    Input("input2", "value"),
)



def update_output(input1, input2):
    specific_x = input1
    intercept = results.params[0]
    intercept_for_specific_x = intercept + results.params[1] * specific_x


    return f'In the year (2022-2045) {input1} if you would sell your house at: ${input2} You will have an expected price differential of: ${round(intercept_for_specific_x,2)}'


app.run(debug=True)